 
<h1 style="text-align: center;">Lag-Llama model inference</h1>


## **Part 1:** Set up

### Part 1.1 Imports

In [2]:
import pickle
import os
import time 
import torch


from data_prep import load_and_preprocess_data, general_revenue_dataframe, split_univariate_data, get_lag_llama_dataset
from config import MODEL_PATH, find_device
from models import lag_llama_estiamtor

Using MPS device
Using MPS device


/Users/nairabarseghyan/anaconda3/lib/python3.11/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


### Part 1.2 Set up lag-llama environment

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%cd lag-llama

### Part 1.3 import lag-llama libraries

In [ ]:
from lag_llama.gluon.estimator import LagLlamaEstimator
from matplotlib import pyplot as plt
from gluonts.evaluation import Evaluator, make_evaluation_predictions

## **Part 2:**  Data

In [ ]:
#loading data and adding necessary features
initial_data = load_and_preprocess_data()
univariate_data = general_revenue_dataframe(initial_data)

train_data_raw, test_data_raw = split_univariate_data(univariate_data)

train_data = get_lag_llama_dataset(train_data_raw,frequency = "1D")
test_data = get_lag_llama_dataset(test_data_raw,frequency = "1D")

In [ ]:
prediction_length = len(test_data_raw)  # prediction length
num_samples = 1000 # sampled from the distribution for each timestep
llama_device = torch.device(find_device())


Using MPS device


## **Part 3:**  Importing model

In [ ]:
lag_llama_model_filename = os.path.join(MODEL_PATH, 'fine_tuned_lag_llama_model.pth')

# Load the predictor from the file
with open(lag_llama_model_filename, 'rb') as f:
    fine_tuned_predictor = pickle.load(f)


## **Part 4:**  Inference

<h3 style="text-align: center;" >================    ZERO SHOT PREDICTION    ================</h3>

In [ ]:
satrt_time_zs = time.time()

forecasts_ZS_val, tss_ZS_val = get_lag_llama_predictions(
    test_data, prediction_length, num_samples
)
print("Zero - Shot inference completed in --- %s seconds ---" % (time.time() - satrt_time_zs))

predction_zero_shots_df = transfrom_lag_llama_predictions(forecasts_ZS_val, tss_ZS_val, prediction_length)

predction_zero_shots_df.to_csv(f"{DATA_PATH}results/Predictions/lag_llama_zero_shot.csv")

<h3 style="text-align: center;" > ================    FINE TUNED MODEL PREDICTION   ================</h3>

In [ ]:
satrt_time_ft = time.time()

forecasts_FT_val, tss_FT_val = get_lag_llama_predictions(
    test_data, prediction_length, num_samples, predictor = fine_tuned_predictor
)


print("Inference of Fine_tuned completed in --- %s seconds ---" % (time.time() - satrt_time_ft))

predction_fine_tune_df = transfrom_lag_llama_predictions(forecasts_FT_val, tss_FT_val, prediction_length)

predction_fine_tune_df.to_csv(f"{DATA_PATH}results/Predictions/lag_llama_fine_tune.csv")

## **Part 5:**  Evaluations

### Part 5.1 Evaluate zero shot

In [ ]:
forecasts_ZS_train, tss_ZS_train = get_lag_llama_predictions(
    train_data, prediction_length, num_samples
)

In [ ]:
evaluate_lag_llama_model(tss_ZS_train, forecasts_ZS_train, tss_ZS_val, forecasts_ZS_val,model_name = 'zero_shot_lag_llama')

### Part 5.1 Evaluate Fine Tuned Model

In [ ]:
forecasts_FT_train, tss_FT_train = get_lag_llama_predictions(
    train_data, prediction_length, num_samples, predictor = fine_tuned_predictor
)

In [ ]:
evaluate_lag_llama_model(tss_FT_train, forecasts_FT_train, tss_FT_val, forecasts_FT_val,model_name = 'fine_tuned_lag_llama')